# Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Loading the CSV Files into DataFrames

In [3]:
#loading each CSV in a separate data frame.
hotels = pd.read_csv('dim_hotels.csv')
rooms = pd.read_csv('dim_rooms.csv')
date_info = pd.read_csv('dim_date.csv')
bookings = pd.read_csv('fact_bookings.csv')
booking_aggregate = pd.read_csv('fact_aggregated_bookings.csv')

# Checking and Cleaning the bookings DataFrame

In [4]:
bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134590 entries, 0 to 134589
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   booking_id         134590 non-null  object 
 1   property_id        134590 non-null  int64  
 2   booking_date       134590 non-null  object 
 3   check_in_date      134590 non-null  object 
 4   checkout_date      134590 non-null  object 
 5   no_guests          134590 non-null  int64  
 6   room_category      134590 non-null  object 
 7   booking_platform   134590 non-null  object 
 8   ratings_given      56683 non-null   float64
 9   booking_status     134590 non-null  object 
 10  revenue_generated  134590 non-null  int64  
 11  revenue_realized   134590 non-null  int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 12.3+ MB


In [5]:
bookings['ratings_given'].fillna(0.0,inplace=True)
bookings = bookings.astype({
    'property_id' : 'str',
    'booking_date' : 'datetime64[ns]',
    'check_in_date' : 'datetime64[ns]',
    'checkout_date' : 'datetime64[ns]',
})

# Checking and Cleaning the hotels DataFrame

In [6]:
hotels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   property_id    25 non-null     int64 
 1   property_name  25 non-null     object
 2   category       25 non-null     object
 3   city           25 non-null     object
dtypes: int64(1), object(3)
memory usage: 928.0+ bytes


In [7]:
hotels = hotels.astype({'property_id' : 'str'})

# Checking and Cleaning the date_info DataFrame

In [8]:
date_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92 entries, 0 to 91
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      92 non-null     object
 1   mmm yy    92 non-null     object
 2   week no   92 non-null     object
 3   day_type  92 non-null     object
dtypes: object(4)
memory usage: 3.0+ KB


In [9]:
date_info['week no'] = date_info['week no'].apply(lambda x: x[2:] if isinstance(x, str) and len(x) > 2 else x)
date_info['week no'] = pd.to_numeric(date_info['week no'], errors='coerce')
date_info.dropna(subset=['week no'], inplace=True)

date_info = date_info.astype({
    'date': 'datetime64[ns]',
    'week no': 'int',
})

C:\Users\RAHUL\AppData\Local\Temp\ipykernel_12800\794615989.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  date_info = date_info.astype({


In [10]:
bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134590 entries, 0 to 134589
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   booking_id         134590 non-null  object        
 1   property_id        134590 non-null  object        
 2   booking_date       134590 non-null  datetime64[ns]
 3   check_in_date      134590 non-null  datetime64[ns]
 4   checkout_date      134590 non-null  datetime64[ns]
 5   no_guests          134590 non-null  int64         
 6   room_category      134590 non-null  object        
 7   booking_platform   134590 non-null  object        
 8   ratings_given      134590 non-null  float64       
 9   booking_status     134590 non-null  object        
 10  revenue_generated  134590 non-null  int64         
 11  revenue_realized   134590 non-null  int64         
dtypes: datetime64[ns](3), float64(1), int64(3), object(5)
memory usage: 12.3+ MB


# Checking and Cleaning the booking_aggregate DataFrame

In [11]:
booking_aggregate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9200 entries, 0 to 9199
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   property_id          9200 non-null   int64 
 1   check_in_date        9200 non-null   object
 2   room_category        9200 non-null   object
 3   successful_bookings  9200 non-null   int64 
 4   capacity             9200 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 359.5+ KB


In [12]:
booking_aggregate=booking_aggregate.astype({'property_id':'str','check_in_date':'datetime64[ns]'})
booking_aggregate.rename(columns={'room_category':'room_id'},inplace=True)

C:\Users\RAHUL\AppData\Local\Temp\ipykernel_12800\2385304555.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  booking_aggregate=booking_aggregate.astype({'property_id':'str','check_in_date':'datetime64[ns]'})


# Merging booking_aggregate with date_info

In [13]:
booking_aggregate = booking_aggregate.merge(date_info,right_on='date',left_on='check_in_date')
booking_aggregate.drop(['day_type','date'],axis=1,inplace=True)

In [14]:
booking_aggregate

,property_id,check_in_date,room_id,successful_bookings,capacity,mmm yy,week no
0,16559,2022-05-01,RT1,25,30,May 22,19
1,19562,2022-05-01,RT1,28,30,May 22,19
2,19563,2022-05-01,RT1,23,30,May 22,19
3,17558,2022-05-01,RT1,13,19,May 22,19
4,16558,2022-05-01,RT1,18,19,May 22,19
...,...,...,...,...,...,...,...
9195,16563,2022-07-31,RT4,13,18,Jul 22,32
9196,16559,2022-07-31,RT4,13,18,Jul 22,32
9197,17558,2022-07-31,RT4,3,6,Jul 22,32
9198,19563,2022-07-31,RT4,3,6,Jul 22,32


# Merging the bookings DataFrame with Other DataFrames

In [15]:
df=bookings.merge(hotels,on='property_id')
df=df.merge(rooms,left_on='room_category',right_on='room_id',)
df=df.merge(date_info,left_on='check_in_date',right_on='date')
df.head()


,booking_id,property_id,booking_date,check_in_date,checkout_date,no_guests,room_category,booking_platform,ratings_given,booking_status,...,revenue_realized,property_name,category,city,room_id,room_class,date,mmm yy,week no,day_type
0,May012216558RT11,16558,2022-04-27,2022-05-01,2022-05-02,3,RT1,direct online,1.0,Checked Out,...,10010,Atliq Grands,Luxury,Delhi,RT1,Standard,2022-05-01,May 22,19,weekend
1,May012216558RT12,16558,2022-04-30,2022-05-01,2022-05-02,2,RT1,others,0.0,Cancelled,...,3640,Atliq Grands,Luxury,Delhi,RT1,Standard,2022-05-01,May 22,19,weekend
2,May012216558RT13,16558,2022-04-28,2022-05-01,2022-05-04,2,RT1,logtrip,5.0,Checked Out,...,9100,Atliq Grands,Luxury,Delhi,RT1,Standard,2022-05-01,May 22,19,weekend
3,May012216558RT14,16558,2022-04-28,2022-05-01,2022-05-02,2,RT1,others,0.0,Cancelled,...,3640,Atliq Grands,Luxury,Delhi,RT1,Standard,2022-05-01,May 22,19,weekend
4,May012216558RT15,16558,2022-04-27,2022-05-01,2022-05-02,4,RT1,direct online,5.0,Checked Out,...,10920,Atliq Grands,Luxury,Delhi,RT1,Standard,2022-05-01,May 22,19,weekend


# Deriving New Features

In [16]:
#Deriving new features from existing features
df['booking_interval'] = df['check_in_date']-df['booking_date']
df['days_stayed'] = df['checkout_date'] - df['check_in_date']
df['booking_interval'] = df['booking_interval'].dt.days
df['days_stayed'] =df.days_stayed.dt.days

# Dropping Unnecessary Columns

In [17]:
#Dropping unnecessary variables
df.drop(['booking_date','checkout_date','room_class','room_category','date'],axis=1,inplace=True)

In [18]:
df.head()

,booking_id,property_id,check_in_date,no_guests,booking_platform,ratings_given,booking_status,revenue_generated,revenue_realized,property_name,category,city,room_id,mmm yy,week no,day_type,booking_interval,days_stayed
0,May012216558RT11,16558,2022-05-01,3,direct online,1.0,Checked Out,10010,10010,Atliq Grands,Luxury,Delhi,RT1,May 22,19,weekend,4,1
1,May012216558RT12,16558,2022-05-01,2,others,0.0,Cancelled,9100,3640,Atliq Grands,Luxury,Delhi,RT1,May 22,19,weekend,1,1
2,May012216558RT13,16558,2022-05-01,2,logtrip,5.0,Checked Out,9100,9100,Atliq Grands,Luxury,Delhi,RT1,May 22,19,weekend,3,3
3,May012216558RT14,16558,2022-05-01,2,others,0.0,Cancelled,9100,3640,Atliq Grands,Luxury,Delhi,RT1,May 22,19,weekend,3,1
4,May012216558RT15,16558,2022-05-01,4,direct online,5.0,Checked Out,10920,10920,Atliq Grands,Luxury,Delhi,RT1,May 22,19,weekend,4,1


In [19]:
booking_aggregate.head()

,property_id,check_in_date,room_id,successful_bookings,capacity,mmm yy,week no
0,16559,2022-05-01,RT1,25,30,May 22,19
1,19562,2022-05-01,RT1,28,30,May 22,19
2,19563,2022-05-01,RT1,23,30,May 22,19
3,17558,2022-05-01,RT1,13,19,May 22,19
4,16558,2022-05-01,RT1,18,19,May 22,19


# Occupancy Rate

In [20]:
#Occupancy rate = No of successful bookings / capacity
booking_aggregate['occupancy_rate'] = round(booking_aggregate['successful_bookings']*100/booking_aggregate['capacity'])
booking_agg=booking_aggregate[['check_in_date','property_id','week no','room_id','successful_bookings','occupancy_rate','capacity']].groupby(
    ['check_in_date','week no','property_id','room_id']).agg({
    'successful_bookings' : 'sum',
    'occupancy_rate' : 'mean',
    'capacity':'max'
})

In [21]:
booking_aggregate.tail()

,property_id,check_in_date,room_id,successful_bookings,capacity,mmm yy,week no,occupancy_rate
9195,16563,2022-07-31,RT4,13,18,Jul 22,32,72.0
9196,16559,2022-07-31,RT4,13,18,Jul 22,32,72.0
9197,17558,2022-07-31,RT4,3,6,Jul 22,32,50.0
9198,19563,2022-07-31,RT4,3,6,Jul 22,32,50.0
9199,17561,2022-07-31,RT4,3,4,Jul 22,32,75.0


# Final Merging and Data Cleaning

In [22]:
booking_aggregate = booking_aggregate.merge(df[['property_id', 'property_name']].drop_duplicates(), on='property_id', how='left')

In [23]:
booking_aggregate.tail()

,property_id,check_in_date,room_id,successful_bookings,capacity,mmm yy,week no,occupancy_rate,property_name
9195,16563,2022-07-31,RT4,13,18,Jul 22,32,72.0,Atliq Palace
9196,16559,2022-07-31,RT4,13,18,Jul 22,32,72.0,Atliq Exotica
9197,17558,2022-07-31,RT4,3,6,Jul 22,32,50.0,Atliq Grands
9198,19563,2022-07-31,RT4,3,6,Jul 22,32,50.0,Atliq Palace
9199,17561,2022-07-31,RT4,3,4,Jul 22,32,75.0,Atliq Blu


In [24]:
missing_values_per_column = booking_aggregate.isnull().sum()

In [25]:
missing_values_per_column

property_id            0
check_in_date          0
room_id                0
successful_bookings    0
capacity               0
mmm yy                 0
week no                0
occupancy_rate         0
property_name          0
dtype: int64

In [26]:
booking_aggregate['is_weekend'] = booking_aggregate['check_in_date'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)

In [27]:
booking_aggregate.head()

,property_id,check_in_date,room_id,successful_bookings,capacity,mmm yy,week no,occupancy_rate,property_name,is_weekend
0,16559,2022-05-01,RT1,25,30,May 22,19,83.0,Atliq Exotica,1
1,19562,2022-05-01,RT1,28,30,May 22,19,93.0,Atliq Bay,1
2,19563,2022-05-01,RT1,23,30,May 22,19,77.0,Atliq Palace,1
3,17558,2022-05-01,RT1,13,19,May 22,19,68.0,Atliq Grands,1
4,16558,2022-05-01,RT1,18,19,May 22,19,95.0,Atliq Grands,1


# Saving Data in csv

In [28]:
booking_aggregate.to_csv('final_data.csv')